In [1]:
# %pip install requests
# %pip install beautifulsoup4
# %pip install selenium
# %pip install pandas
# %pip install geopy
# %pip install geopandas
# %pip install folium
# %pip install openmeteo-requests
# %pip install requests-cache retry-requests numpy pandas
# %pip install timezonefinder
# % pip install seaborn

In [2]:
import os
import time
import requests
import pandas as pd
import geopandas as gpd
import folium
import openmeteo_requests
import requests_cache
import sqlite3
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import json
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from geopy.geocoders import Nominatim
from geopy.geocoders import OpenCage
from folium import Marker
from folium import GeoJson
from retry_requests import retry
from timezonefinder import TimezoneFinder 

Function Breakdown Section 1
---
SQL_Files_to_df
- pulls scraped wine data from an SQL file and converts it to a dataframe

Lat_Long_Coordinates
- takes in a location name and returns a lat long value that is associated with it 

Wine_DataFrame 
- creates a dataframe from the initial wine data that contains unique locations and coordinates

Regions_and_Districts
- takes in global shape files for regions and districts, adds this data to the wine dataframe 

Popular_Wine_Stats
- returns a dataframe with the most popular producer of the most popular winetype in the most popular district and region in the top 8 most popular countries 

In [3]:
def SQL_Files_to_df(file_name, host_folder):
    """
    host_folder: name of the host folder with each of the SQL files to read
    return: DataFrame of the contents of each sql file 
    """
    
    # Connect to the SQLite database 
    conn = sqlite3.connect(os.path.join(host_folder, file_name))

    df = pd.read_sql(f'SELECT * FROM red_wines', conn)

    df.set_index('id', inplace = True)

    df = df.dropna()

    return df

In [4]:
def Lat_Long_Coordinates(location_name):
    """
    name: takes in a location name
    return: returns the lat/long coordinates of the names area
    """
    
    #Uses an OpenCage api_key to filter name through geolocator database
    geolocator = OpenCage(api_key = '74ac6790c4464814b25887115034e579')
    location = geolocator.geocode(location_name)

    if location:
        return location.latitude, location.longitude
    
    else:
        #To grab further data points, uses Nominatim service to filter name through additional geolocator database
        geolocator = Nominatim(user_agent="your_unique_user_agent", timeout=10)
        location = geolocator.geocode(location_name)

        if location:
            return location.latitude, location.longitude
        
        else:
            return None, None

In [5]:
def Wine_DataFrame(raw_wine_data):
    """
    Raw_Wine_Data: takes in a df of all the wines 
    return (): returns a data frame where the indices are unique locations, and columns are #instances of each location, and lat/long coordinates

    """
    #Creates a new column with a combination of region + country from the original dataframe
    raw_wine_data['Locations'] = raw_wine_data['Region'] + ', ' + raw_wine_data['Country']

    #Drops any na values in the location column
    raw_wine_data = raw_wine_data.dropna()

    #Creates a list of unique locations and number of instances of each unique locations
    global_locations = raw_wine_data['Locations'].unique()

    #Creates a data frame with 5 columns: Locations, Location_Instances, Latitude, and Longitude
    complete_wine_data = pd.DataFrame({"Locations" : global_locations,
                       'Location_Instances' : raw_wine_data['Locations'].value_counts()
                       })
    
    complete_wine_data[["Lat","Long"]] = complete_wine_data["Locations"].apply(lambda row: pd.Series(Lat_Long_Coordinates(row)))
        
    #Set index to Locations
    complete_wine_data.set_index('Locations', inplace = True)

    return complete_wine_data

In [6]:
def Regions_and_Districts(wine_data, Location_DataFrame, geojson_host_folder):
    """
    Inputs: The wine data, the intermediate location df created before and a host folder that contains teh geojson data
    Output: A modified wine data df with 3 new columns 
        LatLong_Points: geodataframe geometry point using lat long coords
        Regions: polygon geometery of the region the lat long coords are in 
        District: polygon geometery of the region the lat long coords are in 
    """

    wine_data = wine_data.dropna()


    #Defines the file names for the regional and district files
    regions_filename = 'geoBoundariesCGAZ_ADM1_regions.geojson'
    districts_filename = 'geoBoundariesCGAZ_ADM2_districts.geojson'

    #Adds the lat long coords to the wine dataframe 
    Location_df_changed = Location_DataFrame.drop(columns = ['Location_Instances'])
    wine_data['Locations'] = wine_data['Region'] + ', ' + wine_data['Country']
    wine_data = pd.merge(wine_data, Location_df_changed, on = "Locations", how = 'left')
    wine_data = wine_data.drop('Region', axis = 1)

    #Creates a geodataframe with the coordinates
    wine_data_gdf = gpd.GeoDataFrame(wine_data, geometry = gpd.points_from_xy(x = wine_data['Long'], y = wine_data['Lat']), crs = 'EPSG:4326')
    wine_data_gdf = wine_data_gdf.rename(columns={'geometry': 'LatLong_Points'})
    wine_data_gdf = wine_data_gdf.set_geometry('LatLong_Points')

    #Reads the regional and district geodataframes 
    regions_gdf = gpd.read_file(os.path.join(geojson_host_folder, regions_filename))
    districts_gdf = gpd.read_file(os.path.join(geojson_host_folder, districts_filename))
    
    #Ensure both GeoDataFrames are using the same CRS
    regions_gdf = regions_gdf.to_crs(wine_data_gdf.crs)
    districts_gdf = districts_gdf.to_crs(wine_data_gdf.crs)

    #Joins the wine data df with the regions and districts geodataframs
    wine_data_gdf = gpd.sjoin(wine_data_gdf, regions_gdf[['shapeName', 'geometry']], how='left', op='within')
    wine_data_gdf = gpd.sjoin(wine_data_gdf, districts_gdf[['shapeName', 'geometry']], how='left', op='within', lsuffix='_region', rsuffix='_district')

    #Rename columns for clarity
    wine_data_gdf = wine_data_gdf.rename(columns={
        'shapeName__region' : 'Region', 
        'shapeName__district' : 'District',
    })

    #Drop geodata, wont need to moving forward after locating the regions and districts for each row 
    wine_data_gdf.drop(['url_idx', 'index_right', 'LatLong_Points', 'index__region'], axis = 1, inplace = True)
    
    #Set dataframe order, for clarity
    wine_data_gdf = wine_data_gdf[['WineType','Producer', 'Locations', 'Rating',
                                    'Num_Ratings', 'Price', 'Country', 'Region', 'District',
                                    'Lat', 'Long', 'URL']]
    
    return wine_data_gdf.dropna()

In [7]:
def Popular_Wine_Stats(wine_data_final):
    """
    Input: wine_data_final which should contain a dataframe that has counties, regions, wintetype, and producers
    Output: A dataframe containing the top countries, 
            region within those countries, top 5 districts within that region,
            top winetype within that district and top producer of that wine
    """
    #Creates a simple name for easy use while referencing 
    df = wine_data_final.copy()

    #Get total bottle counts by country and filter out countries with bottle counts more than the 75th percentile
    total_bottles_country = df['Country'].value_counts()
    top_countries = total_bottles_country[total_bottles_country > total_bottles_country.quantile(0.75)]

    #Filter the df to make winetypes uniform 
    popular_wine_types = ['château margaux', 'cabernet sauvignon', 'pinot noir', 'zinfandel', 'syrah', 
                            'pinot gris', 'sauvignon blanc', 'chardonnay', 'baco noir', 'bordeaux',
                            'malbec', 'chardonnay', 'pinot grigio', 'merlot', 'sangiovese', 'shiraz',
                            'cabernet franc', 'muscat', 'grenache', 'sangiovese'  ]
    
    #Sets all the winetypes to be lower 
    df['WineType'] = df['WineType'].str.lower()

    #Filters through each of the winetypes, then changes the df winetype name if the wine type is in the row string 
    for winetype in popular_wine_types:

        df['WineType']  = df['WineType'].apply(lambda row: next((winetype for winetype in popular_wine_types if winetype in row), row))

    #Creates dataframe for output
    popular_wines = pd.DataFrame()

    #------------------------------------------------------------------------------------------------------------#
    #Goes through the top countries and pulls the top regions and the amount of bottles associated with them
    for country in top_countries.index:

        #Checks if region is in top countries, then grabs the top region 
        filter_country_df = df[df['Country'] == country]

        top_region = filter_country_df['Region'].value_counts()
        top_region_count = filter_country_df['Region'].value_counts().max()

        popular_wines = pd.concat([popular_wines, pd.DataFrame([{'Top Country': country, 'Country Count': int(top_countries[country]), 'Top Region': top_region, 'Region Count': top_region_count}])])     
        popular_wines.reset_index(drop = True, inplace = True)

    #------------------------------------------------------------------------------------------------------------#
    #Goes through the top countries and regions within those countries and pulls the top districts and the amount of bottles associated with them
    new_rows = []
    for index, row in popular_wines.iterrows():

        #Checks if winetype is in top region, then grabs the top winetype 
        filter_region_df = df[df['Region'] == row['Top Region']]
        top_district_count = filter_region_df['District'].value_counts().nlargest(5)

        #Goes through each of the top 5 districts and adds the district name and count for a given country and region
        for district, count in top_district_count.items():
            new_rows.append({'index' : index, 'Top District' : district, 'District Count' : count})
        
    #Adds the district data to the popular_wines dataframe 
    district_type = pd.DataFrame(new_rows)
    district_type.set_index('index', inplace = True)

    #Merge district dataframe to the popular wines dateframe
    popular_wines = pd.merge(popular_wines, district_type, left_index = True, right_index = True, how = 'left')

    #------------------------------------------------------------------------------------------------------------#
    # Goes through the top counties and gets top district and region within those countries and pulls the top winetype and the amount of bottles associated with them

    new_rows = []
    for index, row in popular_wines.iterrows():

        #Filter the dataframe based on region and district
        filtered_district_df = df[(df['Region'] == row['Top Region']) & (df['District'] == row['Top District'])]

        #Checks if winetype is in top region, then grabs the top winetype 
        top_winetype = filtered_district_df['WineType'].value_counts().idxmax()
        top_winetype_count = int((filtered_district_df['WineType'].value_counts().max()))
        
        new_rows.append({'Top WineType' : top_winetype, 'WineType Count' : top_winetype_count})
    
    #Adds the winetype data to the popular_wines dataframe 
    wine_type = pd.DataFrame(new_rows)
    popular_wines['Top WineType'] = wine_type['Top WineType'].values
    popular_wines['WineType Count'] = wine_type['WineType Count'].values
    popular_wines.reset_index(inplace = True)

    #------------------------------------------------------------------------------------------------------------#
    #Goes through the top countries, regions within those countries, and top winetype within that region 
    #Pulls the producers of that winetype, and takes a mean of them, returning the top 5 from that mean
    
    new_rows = []
    for index, row in popular_wines.iterrows():

                #Checks if producer is in top region, district, and produces top winetype, then grabs the top producers
                top_producers = df[(df['WineType'] == row['Top WineType']) & (df['District'] == row['Top District']) & (df['Region'] == row['Top Region'])]

                #Grabs the quantiles for the number of ratings, then filters out the data below the lowest 25% 
                #The mean is then found for the ratings based on the producer 
                Num_of_Ratings_Quantile = np.quantile(df['Num_Ratings'], [0.25, 0.5, 0.75])
                top_producer = top_producers[top_producers['Num_Ratings'] >  Num_of_Ratings_Quantile[0]].groupby(by = 'Producer')['Rating'].mean().sort_values(ascending = False)

                new_rows.append({'Producer' : top_producer.index[0], 'Average Rating' : top_producer[0]})

    producer_rating = pd.DataFrame(new_rows)
    popular_wines['Producer'] = producer_rating['Producer'].values
    popular_wines['Average Rating'] = producer_rating['Average Rating'].values
    popular_wines.reset_index(inplace = True)
    #------------------------------------------------------------------------------------------------------------#

    #Add price, URL, and lat/long coords for the outputed points
    df_urllatlong = pd.DataFrame()
    df_urllatlong[['Price', 'Producer','Lat', 'Long', 'URL']] = df.drop_duplicates(subset = 'Producer')[['Price', 'Producer',  'Lat', 'Long', 'URL']]
    final_popular_wines = pd.merge(popular_wines, df_urllatlong, on = 'Producer')
   
    return final_popular_wines.iloc[:, 2:]

---

Variable Section 1
---

In [ ]:
red_wines = SQL_Files_to_df('red_wines_final.db', r'C:\Users\fwhal\Downloads\CME528\Project\BreakinBadCode\Wine_Raw_Data')
location_df = Wine_DataFrame(red_wines)

In [ ]:
red_wines

,Producer,WineType,Year,Region,Country,URL,Rating,Num_Ratings,Price,url_idx,Locations
id,,,,,,,,,,,
1,Antinori,Tenuta Tignanello 'Solaia',2011.0,Toscana,Italy,https://www.vivino.com/antinori-tuscany-tenuta...,4.7,2152,749.99,0,"Toscana, Italy"
2,Château Margaux,Château Margaux (Premier Grand Cru Classé),2006.0,Margaux,France,https://www.vivino.com/chateau-margaux-chateau...,4.7,1883,1274.99,0,"Margaux, France"
3,Antinori,Tenuta Tignanello 'Solaia',2017.0,Toscana,Italy,https://www.vivino.com/antinori-tuscany-tenuta...,4.7,1772,729.99,0,"Toscana, Italy"
4,Schrader,Cabernet Sauvignon Beckstoffer To Kalon Vineyard,2017.0,Oakville,United States,https://www.vivino.com/schrader-cellars-cabern...,4.7,399,509.99,0,"Oakville, United States"
5,Château Margaux,Château Margaux (Premier Grand Cru Classé),2017.0,Margaux,France,https://www.vivino.com/chateau-margaux-chateau...,4.7,391,1374.99,0,"Margaux, France"
...,...,...,...,...,...,...,...,...,...,...,...
23976,Domaine du Ridge,Le Bâtonnier,2022.0,Quebec,Canada,https://www.vivino.com/domaine-du-ridge-le-bat...,3.1,147,18.20,924,"Quebec, Canada"
23977,San Antonio,Sangiovese,2020.0,Rubicone,Italy,https://www.vivino.com/it-san-antonio-sangiove...,3.1,64,14.50,924,"Rubicone, Italy"
23978,Santa Carolina,Gun Club Reservado Cabernet Sauvignon - Merlot,2020.0,Central Valley,Chile,https://www.vivino.com/santa-carolina-gun-club...,3.1,36,18.00,924,"Central Valley, Chile"


In [ ]:
location_df

,Location_Instances,Lat,Long
Locations,,,
"Toscana, Italy",1851,43.458654,11.138920
"Margaux, France",997,45.046334,-0.672912
"Oakville, United States",704,41.099817,-91.044402
"Stags Leap District, United States",691,36.578676,-79.389888
"Rutherford, United States",477,35.399463,-81.902319
...,...,...,...
"Tulum Valley, Argentina",1,-34.000000,-64.000000
"Bendigo, Australia",1,-36.759018,144.282672
"Buzet, France",1,43.709664,-0.831324


In [ ]:
#Long processing time, have saved previous output to csv, load from this
FINAL_wine_df_FINAL = Regions_and_Districts(red_wines, location_df, r'C:\Users\fwhal\Downloads\CME528\Project\GeoJsonFiles')
FINAL_wine_df_FINAL.to_csv(r'C:\Users\fwhal\Downloads\CME528\Project\BreakinBadCode\Final_DataFrames\FINAL_wine_df_FINAL.csv', index = True)

c:\Users\fwhal\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3553: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
c:\Users\fwhal\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3553: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
#Long processing time, have saved previous output to csv, load from this
FINAL_wine_df_filtered_FINAL = Popular_Wine_Stats(FINAL_wine_df_FINAL)
FINAL_wine_df_filtered_FINAL.to_csv(r'C:\Users\fwhal\Downloads\CME528\Project\BreakinBadCode\Final_DataFrames\FINAL_wine_df_filtered_FINAL.csv', index = True)
FINAL_wine_df_filtered_FINAL

Region
Jackson            1856
Sonoma              465
Tulare              436
Mendocino           362
San Luis Obispo     314
                   ... 
New Castle            1
Moniteau              1
Marin                 1
Harris                1
Bingham               1
Name: count, Length: 83, dtype: int64
Region
Toscana                  2430
Piemonte                 2116
Umbria                    579
Sicilia                   211
Veneto                    204
Campania                  129
Abruzzo                    98
Puglia                     91
Sardegna                   76
Trentino-Alto Adige        48
Basilicata                 32
Marche                     25
Emilia-Romagna             24
Lombardia                  24
Lazio                      23
Friuli Venezia Giulia      20
Valle d'Aosta              12
Liguria                    10
Molise                      9
Calabria                    5
Name: count, dtype: int64
Region
Gironde           1733
Côte-d'Or         1120
Vaucl

ValueError: Can only compare identically-labeled Series objects

---